In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.1, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.06355388462543488
epoch:  1, loss: 0.057262152433395386
epoch:  2, loss: 0.052650053054094315
epoch:  3, loss: 0.04714050516486168
epoch:  4, loss: 0.0416850671172142
epoch:  5, loss: 0.04126954451203346
epoch:  6, loss: 0.04087800160050392
epoch:  7, loss: 0.04046721011400223
epoch:  8, loss: 0.04004017636179924
epoch:  9, loss: 0.03952330723404884
epoch:  10, loss: 0.03948298469185829
epoch:  11, loss: 0.0394793264567852
epoch:  12, loss: 0.03947528451681137
epoch:  13, loss: 0.03946743533015251
epoch:  14, loss: 0.039466943591833115
epoch:  15, loss: 0.03926545009016991
epoch:  16, loss: 0.03789546713232994
epoch:  17, loss: 0.036974452435970306
epoch:  18, loss: 0.036449089646339417
epoch:  19, loss: 0.03594626113772392
epoch:  20, loss: 0.0319615975022316
epoch:  21, loss: 0.03195788338780403
epoch:  22, loss: 0.031917694956064224
epoch:  23, loss: 0.02953372523188591
epoch:  24, loss: 0.028581518679857254
epoch:  25, loss: 0.028549907729029655
epoch:  26, loss:

In [6]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0

, loss: 0.6663558483123779
epoch:  1, loss: 0.6427651047706604
epoch:  2, loss: 0.6177672147750854
epoch:  3, loss: 0.6006138920783997
epoch:  4, loss: 0.5790088772773743
epoch:  5, loss: 0.5591574907302856
epoch:  6, loss: 0.5352960228919983
epoch:  7, loss: 0.5129629969596863
epoch:  8, loss: 0.4914267957210541
epoch:  9, loss: 0.46960267424583435
epoch:  10, loss: 0.4477505087852478
epoch:  11, loss: 0.4249090254306793
epoch:  12, loss: 0.40400296449661255
epoch:  13, loss: 0.3837655484676361
epoch:  14, loss: 0.363253116607666
epoch:  15, loss: 0.34323811531066895
epoch:  16, loss: 0.30751436948776245
epoch:  17, loss: 0.29105401039123535
epoch:  18, loss: 0.2733418643474579
epoch:  19, loss: 0.25357678532600403
epoch:  20, loss: 0.2394772320985794
epoch:  21, loss: 0.2242192029953003
epoch:  22, loss: 0.21001604199409485
epoch:  23, loss: 0.19806869328022003
epoch:  24, loss: 0.1884024292230606
epoch:  25, loss: 0.17727850377559662
epoch:  26, loss: 0.1647949367761612
epoch:  27, 